In [1]:
!modal token set --token-id ak-uYihUyljHUMHQvSuKHjxDL --token-secret as-PNcBevbbT2feBCxTHYSjsK

Verifying token against https://api.modal.com
Token verified successfully
Token written to /home/susu/.modal.toml


In [ ]:
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Not connected to a GPU')
# else:
#   print(gpu_info)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# # 更改当前的工作目录到您的项目目录
# import os
# os.chdir('/content/drive/MyDrive/modal_finetune_sql')

# Finetuning LLaMa + Text-to-SQL

This walkthrough shows you how to fine-tune LLaMa-7B on a Text-to-SQL dataset, and then use it for inference against
any database of structured data using LlamaIndex.

**NOTE**: This code is taken and adapted from Modal's `doppel-bot` repo: https://github.com/modal-labs/doppel-bot.
**NOTE**: A lot of the code is contained in the underlying Python scripts in the `src` directory. We definitely encourage you to go and take a look!

### Setup

NOTE: you will need to setup a Modal account + token in order to use this notebook.

In [ ]:
!pip install -r requirements.txt

### Load Training Data for Finetuning LLaMa

We load data from `b-mc2/sql-create-context` on Hugging Face: https://huggingface.co/datasets/b-mc2/sql-create-context.

This dataset consists of tuples of natural language queries, create table statements, and ground-truth SQL queries. This is the dataset that we use to finetune our SQL model.

In [2]:
data_dir = "data_sql"

!modal run src.load_data_sql --data-dir {data_dir}

✓ Initialized. View run at https://modal.com/apps/ap-4aemSFvNEBCd2RzYONJlUM
⠋ Initializing...
⠸ Creating objects...objects...
⠦ Creating objects...
└── ⠸ Creating mount 
    /home/susu/hugging_face_test/demo/test_nlq_to_sql/modal_finetune_sql/src: 
⠏ Creating objects...
└── ⠦ Creating mount 
    /home/susu/hugging_face_test/demo/test_nlq_to_sql/modal_finetune_sql/src: 
⠹ Creating objects...
└── ⠏ Creating mount 
    /home/susu/hugging_face_test/demo/test_nlq_to_sql/modal_finetune_sql/src: 
⠴ Creating objects...
└── ⠹ Creating mount 
    /home/susu/hugging_face_test/demo/test_nlq_to_sql/modal_finetune_sql/src: 
⠇ Creating objects...
└── 🔨 Created mount 
⠹ Creating objects...t_nlq_to_sql/modal_finetune_sql/src
└── 🔨 Created mount 
⠴ Creating objects...t_nlq_to_sql/modal_finetune_sql/src
└── 🔨 Created mount 
⠇ Creating objects...t_nlq_to_sql/modal_finetune_sql/src
├── 🔨 Created mount 
│   /home/susu/hugging_face_test/demo/test_nlq_to_sql/modal_finetune_sql/src
⠙ Creating objects...
├── 🔨 

### Run Finetuning Script

We run our finetuning script on the loaded dataset.
The finetuning script contains the following components:
- We split the dataset into training and validation splits.
- We format each split into input/output tuples of token id's. This means that the labels are the same as inputs (loss signal is measured on full input, not just on the generated portion).
- We use `LoraConfig` from `peft` for efficient fine-tuning.
- We use `transformers.Trainer` to actually run the training process.
- If a valid `WANDB_PROJECT` is specified, along with the relevant secret in Modal, then we will log results to wandb.

We use Modal to spin up an A100 to run our finetuning code.

In [ ]:
data_dir = "data_sql"

!modal run src.finetune_sql --data-dir {data_dir}

✓ Initialized. View run at https://modal.com/apps/ap-nfnMJcSzRpVt8YgYPJwKSM
⠋ Initializing...
⠴ Creating objects...objects...
└── ⠋ Creating mount 
    /home/susu/hugging_face_test/demo/test_nlq_to_sql/modal_finetune_sql/src: 
⠇ Creating objects...
└── ⠸ Creating mount 
    /home/susu/hugging_face_test/demo/test_nlq_to_sql/modal_finetune_sql/src: 
⠹ Creating objects...
└── ⠦ Creating mount 
    /home/susu/hugging_face_test/demo/test_nlq_to_sql/modal_finetune_sql/src: 
⠴ Creating objects...
└── 🔨 Created mount 
⠇ Creating objects...t_nlq_to_sql/modal_finetune_sql/src
└── 🔨 Created mount 
⠙ Creating objects...t_nlq_to_sql/modal_finetune_sql/src
└── 🔨 Created mount 
⠼ Creating objects...t_nlq_to_sql/modal_finetune_sql/src
└── 🔨 Created mount 
⠧ Creating objects...t_nlq_to_sql/modal_finetune_sql/src
└── 🔨 Created mount 
⠋ Creating objects...t_nlq_to_sql/modal_finetune_sql/src
└── 🔨 Created mount 
⠼ Creating objects...t_nlq_to_sql/modal_finetune_sql/src
└── 🔨 Created mount 
⠧ Creating objec

### Evaluation

We provide a basic evaluation script over sample data from `sql-create-context` so that you can see for yourself how well the finetuned model performs vs. the baseline model.

In [ ]:
!modal run src.eval_sql::main

/home/susu/.local/lib/python3.8/site-packages/langchain/__init__.py:34: UserWarning: Importing BasePromptTemplate from langchain root module is no longer supported. Please use langchain.schema.prompt_template.BasePromptTemplate instead.
  warnings.warn(
/home/susu/.local/lib/python3.8/site-packages/langchain/__init__.py:34: UserWarning: Importing PromptTemplate from langchain root module is no longer supported. Please use langchain.prompts.PromptTemplate instead.
  warnings.warn(
✓ Initialized. View run at https://modal.com/apps/ap-xw3puBOtI02Ei6SZwAZZ6g
⠋ Initializing...
⠏ Creating objects...objects...
└── ⠋ Creating mount 
    /home/susu/hugging_face_test/demo/test_nlq_to_sql/modal_finetune_sql/src: 
⠹ Creating objects...
└── ⠸ Creating mount 
    /home/susu/hugging_face_test/demo/test_nlq_to_sql/modal_finetune_sql/src: 
⠴ Creating objects...
└── ⠦ Creating mount 
    /home/susu/hugging_face_test/demo/test_nlq_to_sql/modal_finetune_sql/src: 
⠇ Creating objects...
└── ⠏ Creating mount

### Integrate Model with LlamaIndex

Now that the model is finetuned, the checkpoints and model binary are stored in a model directory (by default it is in `/vol/data_sql`).

We can now use this model in LlamaIndex for text-to-SQL applications.

Specifically, we provide an interface allowing users to define any `sqlite` data file, and then they can run queries over this data file. We first create and dump a sample `cities.db` file containing (city, population, country) tuples. We then run inference over this file.

#### Create sample db

In [ ]:
# create sample
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

In [ ]:
db_file = "cities.db"
engine = create_engine(f"sqlite:///{db_file}")
metadata_obj = MetaData()

In [ ]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)
metadata_obj.create_all(engine)

In [ ]:
# insert sample rows
from sqlalchemy import insert

rows = [
    {"city_name": "多伦多", "population": 2930000, "country": "加拿大"},
    {"city_name": "东京", "population": 13960000, "country": "日本"},
    {"city_name": "芝加哥", "population": 2679000, "country": "美国"},
    {"city_name": "首尔", "population": 9776000, "country": "韩国"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.connect() as connection:
        cursor = connection.execute(stmt)
        connection.commit()

#### Run Inference

In [ ]:
query = "哪个城市的人口最多?"

!modal run src.inference_sql_llamaindex::main --query '{query}' --sqlite-file-path {db_file} --model-dir "data_sql" --use-finetuned-model True

In [ ]:
# you can also choose to run the original (nonfinetuned model) to compare results
# note: it throws an error

# use non-finetuned model
!modal run src.inference_sql_llamaindex::main --query '{query}' --sqlite-file-path {db_file} --model-dir "data_sql" --use-finetuned-model False

### (Optional) Download Model

If you want to download the model weights for your own use, just run the following script.

In [ ]:
from src.download_weights import main

main("out_model", model_dir="data_sql")

In [ ]:
from huggingface_hub import Repository

def upload_model_to_huggingface(model_dir: str, model_id: str):
    repo = Repository(local_dir=model_dir, clone_from=model_id)
    repo.push_to_hub()

# 使用您的 Hugging Face 用户名和模型名称替换 "your-username/your-model-name"
upload_model_to_huggingface("data_sql", "jtjt520j/test_atom_7B_text_to_sql")